In [ ]:
import pandas as pd
import numpy as np
import logging
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from flask import Flask, request, jsonify, render_template
import joblib
from functools import wraps
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import threading
import time

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Generate synthetic risk scoring data
def generate_synthetic_data(n=10000):
    np.random.seed(42)
    data = {
        'user_id': np.arange(n),
        'transaction_amount': np.random.uniform(1, 10000, n),
        'login_attempts': np.random.randint(1, 10, n),
        'failed_transactions': np.random.randint(0, 5, n),
        'account_age': np.random.randint(1, 365, n),
        'suspicious_activity': np.random.choice([0, 1], size=n, p=[0.9, 0.1])
    }
    return pd.DataFrame(data)

# Load and preprocess data
data = generate_synthetic_data()
X = data[['transaction_amount', 'login_attempts', 'failed_transactions', 'account_age']]
y = data['suspicious_activity']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a risk scoring model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Save model
joblib.dump(model, 'risk_scoring_model.pkl')
logging.info("Model saved successfully!")

# Load the model for inference
model = joblib.load('risk_scoring_model.pkl')

# Initialize Flask app
app = Flask(__name__)
alerts = []

def authenticate(f):
    @wraps(f)
    def decorated_function(*args, **kwargs):
        auth = request.headers.get("Authorization")
        if not auth or auth != "Bearer secure_token":
            return jsonify({"message": "Unauthorized"}), 401
        return f(*args, **kwargs)
    return decorated_function

@app.route('/risk_score', methods=['POST'])
@authenticate
def risk_score():
    data = request.get_json()
    features = np.array([data['transaction_amount'], data['login_attempts'], data['failed_transactions'], data['account_age']]).reshape(1, -1)
    risk_prediction = model.predict(features)[0]
    risk_score = model.predict_proba(features)[0][1] * 100  # Convert to percentage
    logging.info(f"Risk score computed for {data}: {risk_score:.2f}%")
    
    if risk_prediction == 1:
        alert_msg = f"ALERT! Suspicious activity detected for user {data['user_id']} with risk score {risk_score:.2f}%"
        alerts.append(alert_msg)
        logging.warning(alert_msg)
    
    return jsonify({'risk_score': round(risk_score, 2), 'suspicious_activity': int(risk_prediction)})

# Initialize Dash app for visualization
dash_app = dash.Dash(__name__, server=app, routes_pathname_prefix='/dashboard/')

dash_app.layout = html.Div([
    html.H1("Risk Scoring Dashboard"),
    dcc.Graph(id='risk-score-graph'),
    dcc.Graph(id='failed-transactions-graph'),
    dcc.Graph(id='login-attempts-graph'),
    html.H2("Real-Time Alerts"),
    html.Ul(id='alert-list'),
    dcc.Interval(id='interval-component', interval=5000, n_intervals=0)
])

@dash_app.callback(
    [Output('risk-score-graph', 'figure'),
     Output('failed-transactions-graph', 'figure'),
     Output('login-attempts-graph', 'figure'),
     Output('alert-list', 'children')],
    [Input('interval-component', 'n_intervals')]
)
def update_graphs(n):
    df_sample = data.sample(50)
    risk_score_fig = {
        'data': [
            {'x': df_sample['user_id'], 'y': df_sample['transaction_amount'], 'type': 'bar', 'name': 'Transaction Amount'},
            {'x': df_sample['user_id'], 'y': df_sample['suspicious_activity'] * 100, 'type': 'bar', 'name': 'Suspicious Activity'}
        ],
        'layout': {'title': 'User Risk Scores'}
    }
    failed_transactions_fig = {
        'data': [{'x': df_sample['user_id'], 'y': df_sample['failed_transactions'], 'type': 'bar', 'name': 'Failed Transactions'}],
        'layout': {'title': 'Failed Transactions per User'}
    }
    login_attempts_fig = {
        'data': [{'x': df_sample['user_id'], 'y': df_sample['login_attempts'], 'type': 'bar', 'name': 'Login Attempts'}],
        'layout': {'title': 'Login Attempts per User'}
    }
    return risk_score_fig, failed_transactions_fig, login_attempts_fig, [html.Li(alert) for alert in alerts[-5:]]

if __name__ == '__main__':
    app.run(debug=True)
